In [ ]:
import requests
import os
import logging
import sys
import json
import cartoframes
logging.basicConfig(stream=sys.stderr, level=logging.INFO)

In [ ]:
CARTO_URL = 'https://{}.carto.com/api/v2/sql'
CARTO_USER = 'wri-rw'
CARTO_KEY = ''
STRICT = True

cc = cartoframes.CartoContext(base_url='https://{}.carto.com/'.format(CARTO_USER),
                              api_key=CARTO_KEY)

In [ ]:
def getTables(user=CARTO_USER, key=CARTO_KEY, f='csv'):
    '''Get the list of tables'''
    r = get('SELECT * FROM CDB_UserTables()', f=f)
    if f == 'csv':
        return r.text.split("\r\n")[1:-1]
    return r

def sendSql(sql, user=CARTO_USER, key=CARTO_KEY, f='', post=True):
    '''Send arbitrary sql and return response object or False'''
    url = CARTO_URL.format(user)
    payload = {
        'api_key': key,
        'q': sql,
    }
    if len(f):
        payload['format'] = f
    logging.debug((url, payload))
    if post:
        r = requests.post(url, json=payload)
    else:
        r = requests.get(url, params=payload)
    if not r.ok:
        logging.error(r.text)
        if STRICT:
            raise Exception(r.text)
        return False
    return r

def get(sql, user=CARTO_USER, key=CARTO_KEY, f=''):
    '''Send arbitrary sql and return response object or False'''
    return sendSql(sql, user, key, f, False)

def post(sql, user=CARTO_USER, key=CARTO_KEY, f=''):
    '''Send arbitrary sql and return response object or False'''
    return sendSql(sql, user, key, f)

def dropTable(table, user=CARTO_USER, key=CARTO_KEY):
    '''Delete table'''
    sql = 'DROP TABLE "{}"'.format(table)
    return post(sql)

In [ ]:
tables = getTables()
# Load config options
try:
    table_info = json.load(open('/Users/nathansuberi/Documents/GitHub/ResourceWatchCode/Metadata Management/table_info.json', 'r'))
except Exception as e:
    logging.info(e)
    table_info = {}

In [ ]:
matches = {}
for name in table_info:
    matches[name] = [table for table in tables if name in table]

In [ ]:
matches

# Switch long-formed to wide tables, save wide tables

In [ ]:
exceptions = ['soc_025', 'country_aliases', 'com_015', 'ene_022', 'cit_022',
             'soc_062', 'wat_005', 'soc_023']
skipped = []
processed = []
for name, tables in matches.items():
    if len(tables)==2:
        if not any([name in tables[1] for name in exceptions]):
            processed.append(name)
#             old_table = tables[0]
#             new_table = tables[1]
#             print('Old table: {}'.format(old_table))
#             print('New table: {}'.format(new_table))
            
#             # Load data
#             wide_data = cc.read(old_table)
#             long_data = cc.read(new_table)
            
#             # Write data
#             print('Writing wide_data to {}'.format(old_table + '_wide'))
#             cc.write(wide_data, old_table + '_wide', overwrite=True)
#             print('Writing long_data to {}'.format(old_table))
#             cc.write(long_data, old_table, overwrite=True)
            
#             # Delete other storage of new data
#             print('Dropping {}'.format(new_table))
#             dropTable(new_table)
        else:
            skipped.append(name)
    else:
        skipped.append(name)
        #print('Second look: {}'.format(name))

In [ ]:
processed

# Delete select tables

In [ ]:
all_to_delete = []
for name, tables in matches.items():
    print(name)
    print(tables)
    for table in tables:
        print(table)
        delete = input('Delete this table? type anything for yes, empty for no')
        if delete:
            all_to_delete.append(table)

In [ ]:
# soc_025_gender_inequality_index

In [ ]:
for table in all_to_delete:
    print('dropping table: ' + table)
    dropTable(table)